In [1]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [2]:

"""
Connect Aliyun and commit MySQL
"""
 
import requests
import json
 
 
def aliq(db, req):
    url = 'http://localhost:8008/aliq?sql=%s&db=%s' % (requests.utils.quote(req), db)
    resp = requests.get(url)
    res = resp.content.decode()
    js = json.loads(res)
    return js

In [3]:
df_wolf = ssql("""SELECT t1.distinct_id,
       t1.`最近一周登录天数`
FROM
  (SELECT distinct_id,
          count(DISTINCT date) AS `最近一周登录天数`
   FROM events
   WHERE event = 'login'
     AND appId in('10001','10002')
     AND date BETWEEN current_date() - interval 7 DAY AND current_date()
   GROUP BY 1 HAVING count(DISTINCT date) >=5) t1
LEFT JOIN
  (SELECT distinct_id
   FROM events
   WHERE event = 'gameStart'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-10-16' AND current_date()
   GROUP BY 1)t2 ON t1.distinct_id = t2.distinct_id
WHERE t2.distinct_id IS NULL
GROUP BY 1,
         2
""")
wolf_user_list = list(df_wolf.distinct_id)
group_user_dict = {str(i) : [] for i in range(100)}
for i in wolf_user_list:
    no = str(int(i[-2:])) # 取出uid的后两位
    group_user_dict[no].append(i) # 将uid按照后两位存入先前构造的字段中
 
all_info = []
for i in range(100):   
    if group_user_dict[str(i)]: # 如果不为空
        table = 'user_phone_%s' % str(i) # 表名
 
        user_list = group_user_dict[str(i)] # uid列表
        if len(user_list) == 1: 
            user_list.append('123')
        sql = """
            select uid, phone_number
            from {table}
            where uid in{user_list}
        """.format(table=table, user_list=tuple(user_list))
        data = aliq('spy', sql)
        if data[0]:
            all_info.extend(data)
df_phone = pd.DataFrame.from_dict([{'distinct_id': str(i[0]), 'phone_number': str(i[1])} for i in all_info])
df_merge = pd.merge(df_wolf, df_phone, on='distinct_id', how='left')
df_merge

,最近一周登录天数,distinct_id,phone_number
0,8,101756040,8619808582019
1,8,89541507,8615800192836
2,5,80866446,8613627097353
3,6,62378089,8619855421297
4,6,105526128,8618252485497
...,...,...,...
113134,8,134684054,8613541309991
113135,6,42975027,NaN
113136,5,119722465,8615304200222
113137,8,135120378,19842352679
